In [9]:
import numpy as np
import pandas as pd
import xgboost as xgb
from config import *
from cal_accuracy import *
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [10]:
# 加载模型并使用
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model.model")

In [11]:
test_data_path = r'./input/80_mfcc_features.xlsx'
# test_data_path = r'./input/BX_mfcc_features.xlsx'
test_data = pd.read_excel(test_data_path)
X_test = test_data[used_feature_columns].values
y_test = test_data['score']
dtest = xgb.DMatrix(X_test)
y_pred = loaded_model.predict(dtest)

In [12]:
y_pred

array([4.1003814, 4.7087326, 3.9116578, 4.505127 , 5.133733 , 4.0449896,
       4.172427 , 2.950767 , 5.2831154, 3.7745504, 3.876462 , 4.6491475,
       5.589925 , 5.190408 , 3.5560749, 4.890977 , 6.135751 , 5.0477805,
       3.7586544, 6.7336183, 6.574779 , 5.8727913, 6.1205044, 6.2080345,
       7.206163 , 6.0729694, 4.8404946, 6.760572 , 5.8355703, 6.421427 ,
       3.8161645, 3.9930525, 3.6973379, 4.5940776, 4.021965 , 5.317329 ,
       3.4087245, 5.175335 , 5.345025 , 6.9156675, 3.9823618, 4.671823 ,
       3.9752512, 4.345393 , 5.1488423, 4.795766 , 4.1689587, 4.6423216,
       5.3163395, 5.0369816, 4.537899 , 5.1613846, 4.9386387, 4.7360015,
       4.3693566, 3.5836813, 4.9916196, 3.7236462, 2.918874 , 4.0112653,
       3.1007504, 4.3613763, 4.9012833, 4.497142 , 4.595325 , 5.1666136,
       4.1041126, 4.564349 , 5.005306 , 2.7229142, 5.0677166, 4.911899 ,
       5.676402 , 4.1380816, 3.845627 , 4.6081586, 5.3279376, 4.785335 ,
       4.8249917, 5.4853077], dtype=float32)

In [13]:
contrast = pd.DataFrame()
contrast['true'] = y_test
contrast['predict'] = y_pred
contrast['difference'] = abs(y_pred - y_test)
contrast['file_name'] = test_data['file_name']
contrast

,true,predict,difference,file_name
0,4.575,4.100381,0.474619,ba1-M301-1731.wav
1,5.100,4.708733,0.391267,ba2-M302-0823.wav
2,5.150,3.911658,1.238342,ba3-M303-0743.wav
3,3.150,4.505127,1.355127,ba4-M304-1030.wav
4,5.125,5.133733,0.008733,ba5-M306-0708.wav
...,...,...,...,...
75,5.925,4.608159,1.316841,ns11-N601-1050.wav
76,5.200,5.327938,0.127938,ns12-N602-0859.wav
77,4.250,4.785335,0.535335,ns13-N603-1527.wav
78,5.750,4.824992,0.925008,ns14-N604-1029.wav


In [14]:
threshold = 1
count = (contrast['difference'] < threshold).sum()
acc = count / len(contrast)
acc

0.7375

In [15]:
avg_diff = contrast['difference'].sum() / len(contrast)
# avg_percent = avg_diff / 8
avg_percent = avg_diff / 5
(1 - avg_percent)

0.8549230571985245

In [16]:
r2 = r2_score(contrast['true'], contrast['predict'])
r2

0.27287242869102735

In [17]:
predict_class_list = []
true_class_list = []
for i in range(len(contrast)):
    predict_class, true_class = classify(contrast.predict[i], contrast.true[i])
    predict_class_list.append(predict_class)
    true_class_list.append(true_class)

acc = accuracy(predict_class_list, true_class_list)
acc

0.8375

In [18]:
mse = mean_squared_error(contrast['predict'], contrast['true'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(contrast['predict'], contrast['true'])
mse, rmse, mae

(0.8270024486986138, 0.9093967498834674, 0.7253847140073777)